### Imports

In [1]:
import pandas as pd
import numpy  as np 
import random
import talib
from lightweight_charts import JupyterChart
from colour import Color

from BrainyBackTest.brain import *
from BrainyBackTest.indicator import *
from BrainyBackTest.trade import *

### Data Cleaning

In [2]:
df = pd.read_csv('./0_RawChartData/Crypto_Binance_BTCUSDT.P_5min.csv')

df['time'] = pd.to_datetime(df['DATE'] + ' ' + df['TIME'])
cols = ['time'] + [col for col in df.columns if col != 'time']
df = df[cols]

df.drop(columns=['DATE', 'TIME', 'TICKVOL'], inplace=True)
df.rename(columns={'time':'time', 'OPEN':'open', 'HIGH':'high', 'LOW':'low', 'CLOSE':'close', 'VOL':'volume', 'SPREAD':'spread'}, inplace=True)

df = df.iloc[:500]
# df = df.head(5000)
df.shape

(500, 7)

### Define the custom indicator

In [3]:
ratioRR  = 10
wantLoss = 1

def My_Custom_OnStart_Function(ind:Indicator ,times:np.array, opens:np.array, highs:np.array, lows:np.array, closes:np.array, spreads:np.array, volumes:np.array):

    ind.set_new_buffer(numBuffer=0, nullValue=0, titleBuffer='ema21', jobBuffer='DrawLine', widthBuffer=1, colorBuffer=Color('green') , windowBuffer='SamePanel')
    ind.set_new_buffer(numBuffer=1, nullValue=0, titleBuffer='ema55', jobBuffer='DrawLine', widthBuffer=1, colorBuffer=Color('orange'), windowBuffer='SamePanel')
    ind.set_new_buffer(numBuffer=2, nullValue=0, titleBuffer='ArrowUps', jobBuffer='DrawArrowUps', widthBuffer=1, colorBuffer=Color('green'), windowBuffer='SamePanel')
    ind.set_new_buffer(numBuffer=3, nullValue=0, titleBuffer='ArrowDns', jobBuffer='DrawArrowDns', widthBuffer=1, colorBuffer=Color('red')  , windowBuffer='SamePanel')
    ind.set_new_buffer(numBuffer=4, nullValue=0, titleBuffer='Trades', jobBuffer='Signal')


    ind[0, :] = talib.EMA(closes, 21)
    ind[1, :] = talib.EMA(closes, 55)
    

    cross_up   = (ind[0, :-1] <= ind[1, :-1]) & (ind[0, 1:] > ind[1, 1:])
    cross_down = (ind[0, :-1] >= ind[1, :-1]) & (ind[0, 1:] < ind[1, 1:])
    cross_up   = np.concatenate([[False], cross_up])
    cross_down = np.concatenate([[False], cross_down])
    ind[2, cross_up]   = 1111
    ind[3, cross_down] = 1111


    for i in range(len(times)):  #process buffer 4

        if (ind[2, i-1]!=0):

            priceOpening   = opens[i]
            priceSL        = ((ind[0, i-2] + ind[1, i-2]) / 2)
            distanceOpenSL = abs(priceOpening-priceSL)
            priceTP        = priceOpening + (distanceOpenSL*ratioRR)
            v              = wantLoss/distanceOpenSL

            newSignal = SignalRequest(action='Market', direction='BuyLong', valueVolume=v, priceSL=priceSL , priceTP=priceTP)

            ind[4, i] = newSignal
        #


        if (ind[3, i-1]!=0):

            priceOpening   = opens[i]
            priceSL        = ((ind[0, i-2] + ind[1, i-2]) / 2)
            distanceOpenSL = abs(priceOpening-priceSL)
            priceTP        = priceOpening - (distanceOpenSL*ratioRR)
            v              = wantLoss/distanceOpenSL

            newSignal = SignalRequest(action='Market', direction='SellShort', valueVolume=v, priceSL=priceSL , priceTP=priceTP)

            ind[4, i] = newSignal
        #
    #
#

ind_crossEMA = Indicator("EMA21_X_EMA55", functionOnStart=My_Custom_OnStart_Function, chartData=df)

In [8]:
ind_crossEMA.get_knowledge().iloc[140:200]

,time,open,high,low,close,volume,spread,EMA21_X_EMA55_ema21,EMA21_X_EMA55_ema55,EMA21_X_EMA55_ArrowUps,EMA21_X_EMA55_ArrowDns,Fingerprint,Action,Dir,Trigger,Open,TP,SL,Vol
140,2021-01-01 11:40:00,29322.22,29353.70,29293.21,29308.14,0,0,29297.444935,29236.087440,0.0,0.0,0,,,0.0,0.0,0.000000,0.000000,0.000000
141,2021-01-01 11:45:00,29306.89,29310.00,29263.06,29306.09,0,0,29298.230850,29238.587531,0.0,0.0,0,,,0.0,0.0,0.000000,0.000000,0.000000
142,2021-01-01 11:50:00,29306.09,29331.60,29281.52,29295.90,0,0,29298.018954,29240.634405,0.0,0.0,0,,,0.0,0.0,0.000000,0.000000,0.000000
143,2021-01-01 11:55:00,29295.91,29344.87,29295.90,29341.99,0,0,29302.016322,29244.254248,0.0,0.0,0,,,0.0,0.0,0.000000,0.000000,0.000000
144,2021-01-01 12:00:00,29342.00,29353.13,29276.03,29280.01,0,0,29300.015747,29245.531239,0.0,0.0,0,,,0.0,0.0,0.000000,0.000000,0.000000
145,2021-01-01 12:05:00,29280.00,29280.01,29214.13,29251.73,0,0,29295.626134,29245.752623,0.0,0.0,0,,,0.0,0.0,0.000000,0.000000,0.000000
146,2021-01-01 12:10:00,29251.73,29312.00,29251.73,29290.02,0,0,29295.116485,29247.333601,0.0,0.0,0,,,0.0,0.0,0.000000,0.000000,0.000000
147,2021-01-01 12:15:00,29290.03,29365.84,29269.15,29361.95,0,0,29301.192260,29251.427044,0.0,0.0,0,,,0.0,0.0,0.000000,0.000000,0.000000
148,2021-01-01 12:20:00,29363.34,29463.48,29361.63,29450.60,0,0,29314.774781,29258.540364,0.0,0.0,0,,,0.0,0.0,0.000000,0.000000,0.000000
149,2021-01-01 12:25:00,29450.61,29492.00,29411.59,29479.99,0,0,29329.794347,29266.449279,0.0,0.0,0,,,0.0,0.0,0.000000,0.000000,0.000000


### .